In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


학습 데이터를 만든다.

In [2]:
# xData -> [공부시간, 과외 시간]
temp = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
print(type(temp)) # 1차원 리스트 배열
print(temp)

# 파이썬 리스트를 넘파이 배열로 만든다.
# array() 함수 사용
xData = np.array(temp)
print(xData)
print(type(xData)) # 2차원 넘파이 배열

<class 'list'>
[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [3]:
# yData -> [합격여부] -> 실제값, 0:불합격, 1 : 합격
temp = [0, 0, 0, 1, 1, 1, 1]
print(type(temp)) # 1차원 리스트 배열
print(temp)

yData = np.array(temp)
print(type(yData)) # 1차원 넘파이 배열
print(yData)

<class 'list'>
[0, 0, 0, 1, 1, 1, 1]
<class 'numpy.ndarray'>
[0 0 0 1 1 1 1]


In [4]:
# reshape() 함수는 데이터는 그대로 유지한 채 배열의 형태(차원, 구조)를 변경한다.
# 1행 7열 넘파이 배열 -> 7행 1열 넘파이 배열로
yData = np.array(temp).reshape(7, 1)
print(type(yData)) # 2차원 넘파이 배열
print(yData)

<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]


xData와 yData를 저장할 placeholder를 만든다.

In [5]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해서 대입되는 넘파이 배열의 찬원을 지정해야 한다.
# shape=[None, 2] : placeholder에 대입되는 넘파이 배열의 행의 개수는 몇개라도 상관없고 열의 개수는 무조건 2개이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

기울기(가중치) a와 y절편(바이어스) b를 임의로 정한다.

In [6]:
# 난수를 2행 1열로 만든다. -> 행렬의 내적을 이용해 처리한다.
a = tf.Variable(tf.random_uniform([2, 1]), dtype=tf.float32)
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('a = {}, b = {}'.format(sess.run(a), sess.run(b)))
print('a1 = {}, a2 = {}, b = {}'.format(sess.run(a)[0], sess.run(a)[1], sess.run(b)))

a = [[0.2027967 ]
 [0.03791726]], b = [0.13367164]
a1 = [0.2027967], a2 = [0.03791726], b = [0.13367164]


시그모이드 방정식, 오차 함수, 경사 하강

In [7]:
# sigmoid() : tensorflow에서 시그모이드 방정식을 계산하는 함수
# matmul() : tensorflow에서 행렬의 내적(곱)을 계산하는 함수

y = tf.sigmoid(tf.matmul(X, a) + b) # 시그모이드 방정식

loss = -tf.reduce_mean(Y * tf.log(y) + (1-Y) * (tf.log(1-y)))

gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

sigmoid() 함수의 실행 결과(y, 예측값)가 0.5이상이면 1을 0.5 미만이면 0을 리턴한다.

In [8]:
sess = tf.Session()
# cast(캐스팅할 데이터, dtype=캐스팅될 데이터 타입) : tensorflow에서 형변환을 하는 함수
predicted = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32)
print(sess.run(predicted)) # [1 2]

[1 2]


In [9]:
# cast() 함수의 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 캐스팅한다.
predicted = tf.cast(0.4 >= 0.5, dtype=tf.float32) # 거짓일 때
print(sess.run(predicted)) # 0.0
predicted = tf.cast(0.4 < 0.5, dtype=tf.float32) # 참일 때
print(sess.run(predicted)) # 1.0

0.0
1.0


sigmoid() 함수를 실행한 예상값을 계산한다.

In [10]:
predicted = tf.cast(y >= 0.5, dtype=tf.float32) # 예측값, sigmoid()함수를 실행한 결과(y)를 0또는 1로 변환한다.

# sigmoid() 함수를 실행해서 얻은 예측값(predicted)이 실제값(yData)과 일치하는 정도(정확도)를 계산한다.
# equal() : tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False를 리턴하는 함수
# equal() 함수로 예측값(predicted)과 실제값(Y)이 같은가 비교한 후
# True 또는 False 결과를 1 또는 0으로 캐스팅하고 시행된 전체 결과의 평균을 계산한다.

accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32)) # 정확도

학습을 시킨다.

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(3001) :
    # sess.run([기울기, y절편, 오차함수, 경사하강], feed_dict={X:입력데이터, Y: 입력데이터에 따른 실제값})
    # 연산을 하긴 하는데 결과를 사용하지 않을 실행 결과를 기억하는 변수는 '_'로 사용한다.
    a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict={X: xData, Y: yData})
    if (i % 300 == 0) :
        # Epoch : 학습 횟수, a1 : 공부 시간 기울기, a2 : 과외 시간 기울기, b : y절편, loss : 오차 함수
        # print('Epoch : %4d, a1 : %9.5f, a2 : %9.5f, b : %9.5f, loss : %9.5f'%(i, a_[0], a_[1], b_, loss_))
        print('Epoch : {0:4d}, a1 : {1:9.5f}, a2 : {2:9.5f}, b : {3:9.5f}, loss : {4:9.5f}'
              .format(i, a_[0][0], a_[1][0], b_[0], loss_))

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
print('=' * 80)

# 테스트 데이터를 만든다.
# new_x = np.array([6, 5]) # 1차원 배열 -> 2차원 배열로
# 테스트 데이터가 1차원 넘파이 배열이기 때문에 행렬의 곱셈 연산이 실행되지 않는다.
# print(new_x.shape) # (2,)
# reshape() 함수로 테스트 데이터를 1행 2열인 2차원 넘파이 배열로 변환해야 행렬의 곱셈 연산이 제대로 실행된다.
new_x = np.array([6, 5]).reshape(1, 2)
# print(new_x) # [[6 5]]

# sigmoid() 함수의 연산 결과에 테스트 데이터를 대입해서 연산한다.
result, new_y = sess.run([predicted, y], feed_dict={X: new_x})
print('공부 시간 : {}, 과외 시간 : {}'.format(new_x[0, 0], new_x[0, 1]))
# print('합격여부 : {} , 합격 확률 : {}%'.format(result[0][0], new_y[0][0]))
print('합격여부 : {0:4s} , 합격 확률 : {1:.3f}%'.format('합격' if (result[0][0] == 1) else '불합격', new_y[0][0]*100))

Epoch :    0, a1 :   0.36924, a2 :   0.04133, b :   0.76194, loss :   1.53559
Epoch :  300, a1 :   0.89955, a2 :  -0.71918, b :  -2.17352, loss :   0.28330
Epoch :  600, a1 :   0.88381, a2 :  -0.41599, b :  -3.69940, loss :   0.20130
Epoch :  900, a1 :   0.78259, a2 :  -0.06614, b :  -4.79982, loss :   0.15621
Epoch : 1200, a1 :   0.67011, a2 :   0.25734, b :  -5.66686, loss :   0.12711
Epoch : 1500, a1 :   0.56426, a2 :   0.54374, b :  -6.38344, loss :   0.10683
Epoch : 1800, a1 :   0.46933, a2 :   0.79502, b :  -6.99439, loss :   0.09194
Epoch : 2100, a1 :   0.38555, a2 :   1.01587, b :  -7.52702, loss :   0.08060
Epoch : 2400, a1 :   0.31185, a2 :   1.21108, b :  -7.99921, loss :   0.07170
Epoch : 2700, a1 :   0.24688, a2 :   1.38488, b :  -8.42336, loss :   0.06455
Epoch : 3000, a1 :   0.18933, a2 :   1.54078, b :  -8.80837, loss :   0.05868
공부 시간 : 6, 과외 시간 : 5
합격여부 : 합격   , 합격 확률 : 50.788%


In [20]:
# 공부는 1시간도 안하고 과외 수업만 0 ~ 10시간을 받았을 때 합격 여부와 합격 확률을 계산한다.
for i in range(11) :
    new_x = np.array([0, i]).reshape(1, 2)
    result, new_y = sess.run([predicted, y], feed_dict={X:new_x})
    print('공부시간 : %d, 과외시간 : %2d'%(new_x[0, 0], new_x[0, 1]), end=' ')
    print('합격여부 : {0:s}, 합격 확률 : {1:6.3f}%'
          .format('합격' if (result[0][0] == 1) else '불합격', new_y[0][0]*100))
    

공부시간 : 0, 과외시간 :  0 합격여부 : 불합격, 합격 확률 :  0.015%
공부시간 : 0, 과외시간 :  1 합격여부 : 불합격, 합격 확률 :  0.070%
공부시간 : 0, 과외시간 :  2 합격여부 : 불합격, 합격 확률 :  0.325%
공부시간 : 0, 과외시간 :  3 합격여부 : 불합격, 합격 확률 :  1.498%
공부시간 : 0, 과외시간 :  4 합격여부 : 불합격, 합격 확률 :  6.628%
공부시간 : 0, 과외시간 :  5 합격여부 : 불합격, 합격 확률 : 24.890%
공부시간 : 0, 과외시간 :  6 합격여부 : 합격, 합격 확률 : 60.737%
공부시간 : 0, 과외시간 :  7 합격여부 : 합격, 합격 확률 : 87.837%
공부시간 : 0, 과외시간 :  8 합격여부 : 합격, 합격 확률 : 97.119%
공부시간 : 0, 과외시간 :  9 합격여부 : 합격, 합격 확률 : 99.369%
공부시간 : 0, 과외시간 : 10 합격여부 : 합격, 합격 확률 : 99.864%
